В лекциях, по классическим методам Process Mining мы рассмотрели, как можно анализировать процессы, основываясь на цифровых следах. Попробуем применить полученные данные для анализа лог-файла. Мы проанализируем последовательность действий в процессе.

In [1]:
#https://stackoverflow.com/questions/36869258/how-to-use-graphviz-with-anaconda-spyder?lq=1

In [24]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

import pandas as pd
import numpy as np 
import pygraphviz as pgv #ЕСли не установлена библиотека, необходимо ее уставновить при помощи PIP
#import graphviz as pgv 

Загружаем данные из лог-файла

In [25]:
f = open('eventlog_v3.csv', 'r')
log = dict()

В файле у нас имеется идентифкатор операции (буквенночисленные значения) , и статусы операции ( значания 20, 30, 40 и.т.д)

Преобразуем лог-файл   

In [26]:
for line in f:
    line = line.strip()
    if len(line) == 0:
        continue
    parts = line.split(',')
    caseid = parts[1]
    task = parts[2]
    if caseid not in log:
        log[caseid] = []
    log[caseid].append(task)
    
f.close()

Создадим структуру - словарь-словарей, где ключами будут статусы по событиям из лог-файла

In [27]:
F = dict()
for caseid in log:
    for i in range(0, len(log[caseid])-1):
        ai = log[caseid][i][0]
        aj = log[caseid][i+1][0]
        if ai not in F:
            F[ai] = dict()
        if aj not in F[ai]:
            F[ai][aj] = 0
        F[ai][aj] += 1

for ai in sorted(F.keys()):
    for aj in sorted(F[ai].keys()):
        print (ai, '->', aj, ':', F[ai][aj])

2 -> 2 : 496573
2 -> 3 : 10068
2 -> 4 : 2096
2 -> 6 : 1743
3 -> 2 : 1125
3 -> 3 : 469812
3 -> 4 : 8339
3 -> 6 : 46
4 -> 2 : 860
4 -> 4 : 39437
4 -> 6 : 1
6 -> 2 : 135
6 -> 6 : 3041


Необходимо при помощи библиотеки pygraphviz выполнить визуализацию структуры словарь-словарей, которая создана выше. (Дописать код ниже)

In [28]:
#from graphviz import Source, render
G = pgv.AGraph(strict=False, directed=True)
#G = pgv.Digraph(strict=False )
G.graph_attr['rankdir'] = 'TD'
G.node_attr['shape'] = 'box'


values = [F[ai][aj] for ai in F for aj in F[ai]]
x_min = min(values)
x_max = max(values)

y_min=1.0
y_max=5.0

for ai in F:
    for aj in F[ai]:
        x= F[ai][aj]
        y=y_min+(y_max-y_min)*float(x-x_min)/float(x_max-x_min)
        G.add_edge(ai,aj,label=F[ai][aj],penwidth=y)


########## необходимо дописать код для построения графа ##########

Сохраним полученный граф в формате .png

In [59]:
#G.draw('graph.jpg', prog='dot')
G.draw('graph.jpg', prog='circo') 
G.draw('file.ps',prog='circo') 

После того, как мы получили граф, попробуем его проанализировать.

Какой путь (ребро графа из одной операции в другую) является самым коротким (наименьшее количество переходов)?
Какой самый частый переход между операциями (наибольшее количество переходов)?